In [1]:
import pandas as pd
import re

In [2]:
chmfile=pd.read_excel('2019 Chromalloy Repair Directory - CFM56-5B.xlsm',None)
#chmfile2=pd.read_excel('Comp_Repair_CFM56-5B-2019_Catalog.xls')

In [3]:
sheets=list(chmfile.keys())

In [18]:
sheets

['Legend',
 'Master Repair Catalog -CFM56-5B',
 'OEM vs DER Summary',
 'Piv OEM vs DER',
 'SQL - tblCapabilityListing',
 'SQL - 2019 OEM CLP',
 'HPC Blades 2D vs 3D',
 'Revisions - Workscopes',
 'Removed - Components']

In [8]:
ChmMaster=chmfile['Master Repair Catalog -CFM56-5B']

In [9]:
ChmMaster.columns=ChmMaster.iloc[0]

In [10]:
ChmMaster=ChmMaster.drop(ChmMaster.index[0:3])

In [11]:
ChmMaster.reset_index(drop=True,inplace=True)

In [12]:
ChmMaster=ChmMaster.iloc[:,0:10]

In [13]:
ChmMaster

,COMPONENTS,TYPICAL MFG. P/N,REPAIR SOURCE,TURN-TIME,2019 REPAIR PRICE,REF #,REPAIR NO. S/B,WORKSCOPE DESCRIPTION,REPAIR TYPE,Indicate if DER Repairs are salvage repairs or OEM equivalent
0,"BLADE, LPC, STAGE 2",338-002-206-0,CTHA,21,42,72-21-02,Standard Repair,Incl. Inspect & OEM Repair 001 & 002,OEM REPAIR,NaN
1,NaN,338-002-207-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,338-002-208-0,NaN,NaN,NaN,NaN,Inspect,Clean & Inspect,OEM REPAIR,NaN
3,NaN,338-002-209-0,NaN,NaN,NaN,NaN,Repair 001,Blend Airfoil,OEM REPAIR,NaN
4,NaN,338-002-218-0,NaN,NaN,NaN,NaN,Repair 002,Replacement of lubricant coating on blade root,OEM REPAIR,NaN
5,NaN,338-002-219-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,338-002-220-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,338-002-221-0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
CatalogEnd=len(ChmMaster)
NotesStart=ChmMaster.index[ChmMaster['COMPONENTS']=='Notes:'].tolist()[0]
if (CatalogEnd>NotesStart):
    CatalogEnd=NotesStart

In [15]:
ChmMaster=ChmMaster[:CatalogEnd]

In [16]:
modelID=sheets[1].split()[-1][1:]
year=2019
cmpny_name='Chromalloy'

In [15]:
#col=['Model ID','Section','Component/Part Name','Part ID','Place of Repair','Part Nos','Repair Code','Repair Description','Comments','Price']

In [16]:
#modelID=chmfile2.columns[2].split()[0]
#year=chmfile2.columns[2].split()[1]
#cmpny_name='GE'

In [17]:
#col_new=['Year','Company Name','Model ID','Section','Part Name','ATA-Capter (Ref No.)','Part ID (Component Code)','Place of Repair','Part Nos','Repair Code','Repair Description','Comments','Repair Type','TAT Days','Price per ea (USD)']
col_new_flags=['Year','Company Name','Model ID','Section','Part Name','ATA-Capter (Ref No.)','Part ID (Component Code)','Place of Repair','Part Nos','Repair Code','Repair Description','Comments','Repair Type','TAT Days','Price per ea (USD)','pack_flag','Service Count']
#ChmData_new_flags=pd.DataFrame(columns=col_new_flags)
#ChmData_new_flags

In [17]:
ChmMaster.fillna('',inplace=True)

c:\programdata\miniconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [132]:
RepairCodeReference=pd.DataFrame(columns=['ATA_No','RepairCode','RepairDescription','RepairType'])

In [21]:
IdxComponent=[i for i in ChmMaster.index if ChmMaster.iloc[i]['COMPONENTS']!='']

In [149]:
comp=19
ATA='xyz2'
partName='Check'
repSite='Here'
partNo='1111111111'
for i in range(IdxComponent[comp],IdxComponent[comp+1]-2):
    price=ChmMaster[ChmMaster.columns[4]][i]
    TAT=ChmMaster[ChmMaster.columns[3]][i]
    repType=ChmMaster[ChmMaster.columns[8]][i]
    repCode=ChmMaster[ChmMaster.columns[6]][i]
    repDesc=ChmMaster[ChmMaster.columns[7]][i]
    if(repDesc=='' or repDesc.lower().startswith('optional') or repType.lower().startswith('der') or repType.lower().startswith('life-x')):
        pass
    elif(str(price)=='' and str(TAT)=='' and repType.lower()=='oem repair'):
        #print(i,True)
        if repCode not in RepairCodeReference.loc[RepairCodeReference['ATA_No']==ATA]['RepairCode'].unique().tolist():
            RepairCodeReference=RepairCodeReference.append(pd.Series([ATA,repCode,repDesc,repType],['ATA_No','RepairCode','RepairDescription','RepairType']),ignore_index=True)
    else:
    #repType=ChmMaster[ChmMaster.columns[8]][i]
        if(repDesc.lower().startswith('incl')):
            pack_flag=1
            serviceCount=len(parseRepDesc(repDesc))+1
        else:
            pack_flag=0
            serviceCount=1
        dataline=[year,cmpny_name,modelID,'',partName,ATA,'',repSite,partNo,repCode,repDesc,'',TAT,repType,price,pack_flag,serviceCount]
        #print(price,TAT,repCode,repDesc,repType)
        print(dataline)

[2019, 'Chromalloy', 'CFM56-5B', '', 'Check', 'xyz2', '', 'Here', '1111111111', 'Standard Repair', 'Incl. Inspect & OEM Repair 007 & 010', '', 21, 'OEM REPAIR', 1112, 1, 3]
[2019, 'Chromalloy', 'CFM56-5B', '', 'Check', 'xyz2', '', 'Here', '1111111111', 'Insp/Chk', 'Visual, Dimensional and NDT Inspection of Fan Outlet Guide Vane', '', '', 'OEM REPAIR', 75, 0, 1]


In [129]:
def parseRepDesc(repDesc):
    #my_list=ChmMaster[ChmMaster.columns[7]][IdxComponent[6]].split()
    my_list=repDesc.split()
    my_list = [s for s in my_list if re.search(r'\d',s)]
    my_list=[s.replace(',','') for s in my_list]
    for i in range(len(my_list)):
        if re.search(r'-',my_list[i]):
            my_list.extend(["%03d" % a for a in range(int(my_list[i].split('-')[0][-1]),int(my_list[i].split('-')[1][-1])+1)])
            my_list.remove(my_list[i])
            #print(int(my_list[i].split('-')[0][-1]),int(my_list[i].split('-')[1][-1])+1)
    return my_list

In [150]:
RepairCodeReference

,ATA_No,RepairCode,RepairDescription,RepairType
0,xyz2,Repair 001,Removal of Surface Defects and Spot Touch-Up P...,OEM REPAIR
1,xyz2,Repair 002,Replacement of Crimped Self-Locking Nuts on th...,OEM REPAIR
2,xyz2,Repair 003,Replacement of the Anti-Erosion Paint and the ...,OEM REPAIR
3,xyz2,Repair 017,Repair of the Rear Cone Diameter that locates ...,OEM REPAIR
4,xyz2,Repair 019,Replacement of the Bonded Seal on the Spinner ...,OEM REPAIR
5,xyz2,Repair 025,Repair of the Spinner Rear Cone Surface Defects,OEM REPAIR
6,xyz2,Repair 026,Replacement of the Swaged Self-Locking Inserts,OEM REPAIR
7,xyz2,Repair 007,Replacement of Metallic Leading Edge,OEM REPAIR
8,xyz2,Repair 010,Replacement of the Outer Platform,OEM REPAIR


In [ ]:
#New Formatting for Packages & services along with flags
ChmData_new_flags=pd.DataFrame(columns=col_new_flags)
IdxComponent=[i for i in ChmMaster.index if ChmMaster.iloc[i]['COMPONENTS']!='']
for i in IdxComponent:
    #partID=chm[chm.columns[0]][i]
    partName=ChmMaster[ChmMaster.columns[0]][i]
    repsite=ChmMaster[ChmMaster.columns[2]][i]
    ATA_No=ChmMaster[ChmMaster.columns[5]][i]
    TAT=ChmMaster[ChmMaster.columns[3]][i]
    k=i
    partNos=[]
    while(ChmMaster[ChmMaster.columns[1]][k]!=''):
        partNos.append(ChmMaster[ChmMaster.columns[1]][k])
        k+=1
    #i+=1
    for partNo in partNos:
        for
    while(chm[chm.columns[1]][k]!=''):
        pNo=chm[chm.columns[1]][k]
        l=i
        #print(pNo)
        while(True):
            #print('in')
            if chm[chm.columns[3]][l]=='' and chm[chm.columns[4]][l]=='':
                break
            if chm[chm.columns[4]][l].startswith('*'):
                ChmData_new_flags['Comments'][len(ChmData_new_flags)-1]=chm[chm.columns[4]][l]
            elif chm[chm.columns[3]][l]=='' and chm[chm.columns[4]][l]!='':
                repcode=chm[chm.columns[4]][l]
                repDesc=''
                reptype=''
                tatDays=re.sub(r'[+]', '',chm[chm.columns[5]][l])
                price=re.sub(r'[+]', '',chm[chm.columns[6]][l])
                count=0
                l+=1
                if chm[chm.columns[4]][l].startswith('*'):
                    ChmData_new_flags['Comments'][len(ChmData_new_flags)-1]=chm[chm.columns[4]][l]
                    l+=1
                while(chm[chm.columns[3]][l]!=''):
                    repc=chm[chm.columns[3]][l]
                    rept='Non-OEM'
                    if re.match(r'^R',repc):
                        repc=repc[1:]
                        rept='OEM'
                    repDesc+=repc+' : '+chm[chm.columns[4]][l]+'\n'
                    reptype+=rept+'\n'
                    count+=1
                    l+=1    
                pck_flag=1
                dataline=[year,cmpny_name,modelID,section,partName,ATA_No,partID,repSite,pNo,repcode,repDesc,'',reptype,tatDays,price,pck_flag,count]
                ChmData_new_flags=ChmData_new_flags.append(pd.Series(dataline,col_new_flags),ignore_index=True)
                l-=1
            else:
                repcode=chm[chm.columns[3]][l]
                reptype='Non-OEM'
                repDesc=repcode+' : '+chm[chm.columns[4]][l]
                pck_flag=0
                if re.match(r'^R',repcode):
                    repcode=repcode[1:]
                    reptype='OEM'
                dataline=[year,cmpny_name,modelID,section,partName,ATA_No,partID,repSite,pNo,repcode,repDesc,'',reptype,re.sub(r'[+]', '',chm[chm.columns[5]][l]),re.sub(r'[+]', '',chm[chm.columns[6]][l]),pck_flag,1]
                ChmData_new_flags=ChmData_new_flags.append(pd.Series(dataline,col_new_flags),ignore_index=True)
            l+=1
            if(l==len(chm)):
                break
        k+=1
        #print(k)
        if(k==len(chm)):
            break

In [72]:
#New Formatting for Packages & services along with flags
ChmData_new_flags=pd.DataFrame(columns=col_new_flags)
for n in range(0,len(sheets)):
    chm=chmfile[sheets[n]]
    section=sheets[n]
    chm.columns=chm.iloc[0]
    chm=chm.drop(0,axis=0)
    chm=chm.reset_index(drop=True)
    chm=chm.fillna('')
    CompIdx=[i for i in chm.index if chm.iloc[i]['Component Code']!='']
    for i in CompIdx:
        partID=chm[chm.columns[0]][i]
        partName=chm[chm.columns[1]][i]
        repSite=chm[chm.columns[2]][i]
        ATA_No=chm[chm.columns[3]][i]
        k=i+1
        i+=1
        
        while(chm[chm.columns[1]][k]!=''):
            pNo=chm[chm.columns[1]][k]
            l=i
            #print(pNo)
            while(True):
                #print('in')
                if chm[chm.columns[3]][l]=='' and chm[chm.columns[4]][l]=='':
                    break
                if chm[chm.columns[4]][l].startswith('*'):
                    ChmData_new_flags['Comments'][len(ChmData_new_flags)-1]=chm[chm.columns[4]][l]
                elif chm[chm.columns[3]][l]=='' and chm[chm.columns[4]][l]!='':
                    repcode=chm[chm.columns[4]][l]
                    repDesc=''
                    reptype=''
                    tatDays=re.sub(r'[+]', '',chm[chm.columns[5]][l])
                    price=re.sub(r'[+]', '',chm[chm.columns[6]][l])
                    count=0
                    l+=1
                    if chm[chm.columns[4]][l].startswith('*'):
                        ChmData_new_flags['Comments'][len(ChmData_new_flags)-1]=chm[chm.columns[4]][l]
                        l+=1
                    while(chm[chm.columns[3]][l]!=''):
                        repc=chm[chm.columns[3]][l]
                        rept='Non-OEM'
                        if re.match(r'^R',repc):
                            repc=repc[1:]
                            rept='OEM'
                        repDesc+=repc+' : '+chm[chm.columns[4]][l]+'\n'
                        reptype+=rept+'\n'
                        count+=1
                        l+=1    
                    pck_flag=1
                    dataline=[year,cmpny_name,modelID,section,partName,ATA_No,partID,repSite,pNo,repcode,repDesc,'',reptype,tatDays,price,pck_flag,count]
                    ChmData_new_flags=ChmData_new_flags.append(pd.Series(dataline,col_new_flags),ignore_index=True)
                    l-=1
                else:
                    repcode=chm[chm.columns[3]][l]
                    reptype='Non-OEM'
                    repDesc=repcode+' : '+chm[chm.columns[4]][l]
                    pck_flag=0
                    if re.match(r'^R',repcode):
                        repcode=repcode[1:]
                        reptype='OEM'
                    dataline=[year,cmpny_name,modelID,section,partName,ATA_No,partID,repSite,pNo,repcode,repDesc,'',reptype,re.sub(r'[+]', '',chm[chm.columns[5]][l]),re.sub(r'[+]', '',chm[chm.columns[6]][l]),pck_flag,1]
                    ChmData_new_flags=ChmData_new_flags.append(pd.Series(dataline,col_new_flags),ignore_index=True)
                l+=1
                if(l==len(chm)):
                    break
            k+=1
            #print(k)
            if(k==len(chm)):
                break

In [ ]:
ChmData=pd.DataFrame(columns=col)
for n in range(0,len(sheets)):
    chm=chmfile[sheets[n]]
    section=sheets[n]
    chm.columns=chm.iloc[0]
    chm=chm.drop(0,axis=0)
    chm=chm.reset_index(drop=True)
    chm=chm.fillna('')
    CompIdx=[i for i in chm.index if chm.iloc[i]['Component Code']!='']
    for i in CompIdx:
        partID=chm[chm.columns[0]][i]
        partName=chm[chm.columns[1]][i]
        repSite=chm[chm.columns[2]][i]
        k=i+1
        i+=1
        
        while(chm[chm.columns[1]][k]!=''):
            pNo=chm[chm.columns[1]][k]
            l=i
            #print(pNo)
            while(True):
                #print('in')
                if chm[chm.columns[3]][l]=='' and chm[chm.columns[4]][l]=='':
                    break
                if chm[chm.columns[4]][l].startswith('*'):
                    ChmData['Comments'][len(ChmData)-1]=chm[chm.columns[4]][l]
                else:
                    dataline=[modelID,section,partName,partID,repSite,pNo,chm[chm.columns[3]][l],chm[chm.columns[4]][l],'',chm[chm.columns[6]][l]]
                    ChmData=ChmData.append(pd.Series(dataline,col),ignore_index=True)
                l+=1
                if(l==len(chm)):
                    break
            k+=1
            #print(k)
            if(k==len(chm)):
                break

In [38]:
#New Formatting according to the new tempalte
ChmData_new=pd.DataFrame(columns=col_new)
for n in range(0,len(sheets)):
    chm=chmfile[sheets[n]]
    section=sheets[n]
    chm.columns=chm.iloc[0]
    chm=chm.drop(0,axis=0)
    chm=chm.reset_index(drop=True)
    chm=chm.fillna('')
    CompIdx=[i for i in chm.index if chm.iloc[i]['Component Code']!='']
    for i in CompIdx:
        partID=chm[chm.columns[0]][i]
        partName=chm[chm.columns[1]][i]
        repSite=chm[chm.columns[2]][i]
        ATA_No=chm[chm.columns[3]][i]
        k=i+1
        i+=1
        
        while(chm[chm.columns[1]][k]!=''):
            pNo=chm[chm.columns[1]][k]
            l=i
            #print(pNo)
            while(True):
                #print('in')
                if chm[chm.columns[3]][l]=='' and chm[chm.columns[4]][l]=='':
                    break
                if chm[chm.columns[4]][l].startswith('*'):
                    ChmData_new['Comments'][len(ChmData_new)-1]=chm[chm.columns[4]][l]
                else:
                    repcode=chm[chm.columns[3]][l]
                    reptype='Non-OEM'
                    if re.match(r'^R',repcode):
                        repcode=repcode[1:]
                        reptype='OEM'
                    dataline=[cmpny_name,modelID,section,partName,ATA_No,partID,repSite,pNo,repcode,chm[chm.columns[4]][l],'',reptype,re.sub(r'[+]', '',chm[chm.columns[5]][l]),re.sub(r'[+]', '',chm[chm.columns[6]][l])]
                    ChmData_new=ChmData_new.append(pd.Series(dataline,col_new),ignore_index=True)
                l+=1
                if(l==len(chm)):
                    break
            k+=1
            #print(k)
            if(k==len(chm)):
                break

In [79]:
ChmData_new_flags.iloc[753]

Year                                          2019
Company Name                                    GE
Model ID                                  CFM56-5B
Section                             Combustor, HPT
Part Name                      Rear Stator Support
ATA-Capter (Ref No.)                      72-41-07
Part ID (Component Code)                       332
Place of Repair                               ACSC
Part Nos                                1993M20G01
Repair Code                                    001
Repair Description          R001 : Restore flanges
Comments                                          
Repair Type                                    OEM
TAT Days                                        21
Price per ea (USD)                            4565
pack_flag                                        0
Service Count                                    1
Name: 753, dtype: object

In [80]:
ChmData_new_flags.to_excel('GE_'+modelID+'_extracted_Data_flags_3.xlsx',index = None)

In [110]:
for i in range(int('001'),int('002')):
    print(i)

1
